In [4]:
# Importing modules.
import numpy as np

import pandas as pd
from IPython.display import display

from sqlalchemy import create_engine
from sqlalchemy.types import DateTime, VARCHAR
import mysql.connector as connector


In [5]:
# Importing data.
visitorlogs = pd.read_csv(r'D:\Hackathon\Analytics Vidhya\Job-a-thon\data\Modified\VisitorLogsData.csv')
print('Shape of VisitorLogsData', visitorlogs.shape)
display(visitorlogs.head())

user = pd.read_csv(r'D:\Hackathon\Analytics Vidhya\Job-a-thon\data\Modified\userTable.csv')
print('Shape of userTable', user.shape)
display(user.head())


Shape of VisitorLogsData (6588000, 9)


,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


Shape of userTable (34050, 3)


,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [6]:
# Renaming for ease.
user.rename(columns={'Signup Date': 'SignupDate',
                     'User Segment': 'Segment'}, inplace=True)
user.head()

,UserID,SignupDate,Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B
2,U109654,2013-03-19 11:38:55+00:00,B
3,U108998,2018-01-18 08:29:51.627954+00:00,C
4,U131393,2018-03-27 08:05:28.806800+00:00,B


In [7]:
# Dropping unregistered users.
visitorlogs.dropna(subset=['UserID'], inplace=True)

# Cross checking for nulls.
visitorlogs.isnull().sum()

webClientID           0
VisitDateTime     64803
ProductID         51963
UserID                0
Activity          87757
Browser               0
OS                    0
City             236124
Country           42827
dtype: int64

In [8]:
print('Shape of the data is:', visitorlogs.shape)

Shape of the data is: (650695, 9)


#### Cleaning VisitDateTime.

In [9]:
def datetime_clean(date_time):
    '''
    Converts the strings to datetime format.
    input = Date time in string format.
    output = returns datetime value.
    '''

    try:
        return pd.to_datetime(pd.Timestamp(date_time), utc=True)
    except:
        try:
            date_time = int(date_time)
            return pd.to_datetime(pd.Timestamp(date_time), utc=True)
        except:
            pass


In [10]:

# Transforming the 'VisitDateTime' column.
visitorlogs['VisitDateTime'] = visitorlogs['VisitDateTime'].map(lambda x: datetime_clean(x))
user['SignupDate'] = user['SignupDate'].map(lambda x: datetime_clean(x))


In [11]:

# Replacing spaces with 'NaN'.
visitorlogs = visitorlogs.replace(r'^\s*$', np.nan, regex=True)
user = user.replace(r'^\s*$', np.nan, regex=True)


 'VisitDateTime' Imputation.

In [12]:

# Imputing values with mean for each User.
visitorlogs['VisitDateTime'] = visitorlogs.groupby(['UserID'])['VisitDateTime']\
    .transform(lambda x: x.fillna(x.mean()))


In [13]:

# Cross checking for null values
visitorlogs.isnull().sum()


webClientID           0
VisitDateTime       805
ProductID         51963
UserID                0
Activity          87757
Browser               0
OS                    0
City             253493
Country           60196
dtype: int64

 MySQL

In [15]:
# Establishing SQL connection.
connection = connector.connect(host = 'localhost', passwd = 'param12345', 
                       user = 'root')


In [16]:
# Establishing SQL connection.
connection = connector.connect(host = 'localhost', passwd = 'param12345', 
                       user = 'root')



In [17]:
# Creating a cursor.
mycursor = connection.cursor()


In [18]:
# Dropping existing database.
mycursor.execute("DROP DATABASE IF EXISTS data")


In [19]:
# Creating a database.
mycursor.execute("CREATE DATABASE data")

In [21]:

sql_engine = create_engine("mysql+mysqlconnector://" + 'root' + ":"
                       + 'param12345' + "@" + 'localhost' + "/" + 'data')


In [22]:
# Tables.
query = 'Show tables from data'
tables = pd.read_sql_query(query, connection)
tables


,Tables_in_data


 Creating table.

In [23]:
# Declaring variable types.
dtype = {'webClientID': VARCHAR(50),
         'UserID': VARCHAR(50),
         'ProductID': VARCHAR(50),
         'VisitDateTime': DateTime,
         'Activity': VARCHAR(50),
         'OS': VARCHAR(50),
         'Browser': VARCHAR(50),
         'City': VARCHAR(100),
         'Country': VARCHAR(100)}

In [25]:

# Exporting to SQL.
visitorlogs.to_sql(name = 'visitorlogsdata', con = sql_engine,
                   if_exists = 'replace', index = False, dtype = dtype)

In [26]:

# Declaring variable types.
dtype = {'UserID': VARCHAR(10),
         'SignupDate': DateTime,
         'Segment': VARCHAR(5)}

In [27]:

# Exporting to SQL.
user.to_sql(name = 'usertable', con = sql_engine, if_exists = 'replace',
            index = False)


In [29]:
# Update
connection = connector.connect(host = 'localhost', user = 'root', passwd = 'param12345',
                database = 'data')


In [30]:
# Update
connection = connector.connect(host = 'localhost', user = 'root', passwd = 'param12345',
                database = 'data')



In [31]:
# Datas only upto 27th July 2018.

query = "select @current_date:= STR_TO_DATE('2018-05-27 23:59:59','%Y-%m-%d %H:%i:%s.%f')"
max_date = pd.read_sql_query(query, connection)
max_date


,"@current_date:= STR_TO_DATE('2018-05-27 23:59:59','%Y-%m-%d %H:%i:%s.%f')"
0,2018-05-27 23:59:59.000000


In [32]:
# Creating a data frame.
results_data = pd.DataFrame()

In [33]:

# 1) UserID.
results_data['UserID'] = sorted(user.UserID)
results_data


,UserID
0,U100002
1,U100003
2,U100004
3,U100005
4,U100006
...,...
34045,U136960
34046,U136961
34047,U136963
34048,U136964


In [34]:
# 2) No_of_days_Visited_7_Days.
query = 'select UserID, count(distinct day(VisitDateTime)) as No_of_days_Visited_7_Days\
        from visitorlogsdata\
        where VisitDateTime>@current_date-interval 7 day\
        and UserID is not null\
        group by UserID\
        order by UserID\
        ;'

No_of_days_Visited_7_Days = pd.read_sql_query(query, connection)
No_of_days_Visited_7_Days

,UserID,No_of_days_Visited_7_Days
0,U100003,1
1,U100004,1
2,U100005,1
3,U100006,1
4,U100008,7
...,...,...
16167,U119707,1
16168,U119708,1
16169,U119709,1
16170,U119711,1


In [35]:

# Merging.
results_data['No_of_days_Visited_7_Days'] = pd.merge(results_data, No_of_days_Visited_7_Days,
                                                     how = 'left', on = 'UserID')['No_of_days_Visited_7_Days']
results_data


,UserID,No_of_days_Visited_7_Days
0,U100002,NaN
1,U100003,1.0
2,U100004,1.0
3,U100005,1.0
4,U100006,1.0
...,...,...
34045,U136960,NaN
34046,U136961,NaN
34047,U136963,NaN
34048,U136964,NaN


In [36]:
# 3) No_Of_Products_Viewed_15_Days.
query = 'select UserID,count(distinct ProductID) as No_Of_Products_Viewed_15_Days\
        from visitorlogsdata\
        where VisitDateTime> @current_date -interval 15 day\
        and UserID is not null\
        group by UserID\
        order by UserID\
        ;'

No_Of_Products_Viewed_15_Days = pd.read_sql_query(query, connection)
No_Of_Products_Viewed_15_Days


,UserID,No_Of_Products_Viewed_15_Days
0,U100002,2
1,U100003,2
2,U100004,15
3,U100005,4
4,U100006,1
...,...,...
25880,U129368,1
25881,U129369,1
25882,U129370,1
25883,U129371,3


In [37]:
# Merging.
results_data['No_Of_Products_Viewed_15_Days'] = pd.merge(results_data, No_Of_Products_Viewed_15_Days,
                                                         how = 'left', on = 'UserID')['No_Of_Products_Viewed_15_Days']
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days
0,U100002,NaN,2.0
1,U100003,1.0,2.0
2,U100004,1.0,15.0
3,U100005,1.0,4.0
4,U100006,1.0,1.0
...,...,...,...
34045,U136960,NaN,NaN
34046,U136961,NaN,NaN
34047,U136963,NaN,NaN
34048,U136964,NaN,NaN


In [38]:
# 4) User_Vintage.
query = 'select UserID,datediff(@current_date,SignupDate) as User_Vintage\
        from usertable\
        where UserID is not null\
        order by UserID\
        ;'

User_Vintage = pd.read_sql_query(query, connection)
User_Vintage


,UserID,User_Vintage
0,U100002,52
1,U100003,1020
2,U100004,340
3,U100005,680
4,U100006,54
...,...,...
34045,U136960,754
34046,U136961,59
34047,U136963,1419
34048,U136964,494


In [39]:
# Mering.
results_data['User_Vintage'] = pd.merge(results_data, User_Vintage,
                                        how = 'left', on = 'UserID')['User_Vintage']
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage
0,U100002,NaN,2.0,52
1,U100003,1.0,2.0,1020
2,U100004,1.0,15.0,340
3,U100005,1.0,4.0,680
4,U100006,1.0,1.0,54
...,...,...,...,...
34045,U136960,NaN,NaN,754
34046,U136961,NaN,NaN,59
34047,U136963,NaN,NaN,1419
34048,U136964,NaN,NaN,494


In [40]:
# 5) Most_Viewed_product_15_Days.
query = 'select UserID,Most_Viewed_product_15_Days ,max(occurs) as products_viewed\
        from\
        (\
        select UserID,ProductID as Most_Viewed_product_15_Days,count(ProductID) as occurs,VisitDateTime\
            from visitorlogsdata\
            where VisitDateTime>@current_date-interval 15 day\
            and UserID is not null\
            group by UserID,ProductID\
            order by UserID,occurs desc,VisitDateTime desc\
        ) as occur_table\
        group by UserID\
        ;'

Most_Viewed_product_15_Days = pd.read_sql_query(query, connection)
Most_Viewed_product_15_Days


,UserID,Most_Viewed_product_15_Days,products_viewed
0,U100002,pr100258,4
1,U100003,Pr100079,2
2,U100004,Pr100355,4
3,U100005,Pr100234,2
4,U100006,Pr101111,1
...,...,...,...
25880,U129368,pr100071,2
25881,U129369,Pr100507,2
25882,U129370,Pr100739,1
25883,U129371,pr100051,4


In [41]:
# Merging.
results_data['Most_Viewed_product_15_Days'] = pd.merge(results_data, Most_Viewed_product_15_Days,
                                                       how = 'left', on = 'UserID')['Most_Viewed_product_15_Days']
results_data

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days
0,U100002,NaN,2.0,52,pr100258
1,U100003,1.0,2.0,1020,Pr100079
2,U100004,1.0,15.0,340,Pr100355
3,U100005,1.0,4.0,680,Pr100234
4,U100006,1.0,1.0,54,Pr101111
...,...,...,...,...,...
34045,U136960,NaN,NaN,754,NaN
34046,U136961,NaN,NaN,59,NaN
34047,U136963,NaN,NaN,1419,NaN
34048,U136964,NaN,NaN,494,NaN


In [42]:

# Filling NaN values with Product101.
results_data['Most_Viewed_product_15_Days'].fillna('Product101', inplace = True)
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days
0,U100002,NaN,2.0,52,pr100258
1,U100003,1.0,2.0,1020,Pr100079
2,U100004,1.0,15.0,340,Pr100355
3,U100005,1.0,4.0,680,Pr100234
4,U100006,1.0,1.0,54,Pr101111
...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101
34046,U136961,NaN,NaN,59,Product101
34047,U136963,NaN,NaN,1419,Product101
34048,U136964,NaN,NaN,494,Product101


In [43]:
# 6) Most_Active_OS.
query = 'select UserID,OS as Most_Active_OS\
        from\
        (\
        select UserID,OS,count(OS) as occur\
        from visitorlogsdata\
        where UserID is not null\
        group by UserID,OS\
        order by UserID,occur desc\
        ) as occur_table\
        group by UserID;\
        '

Most_Active_OS = pd.read_sql_query(query, connection)
Most_Active_OS


,UserID,Most_Active_OS
0,U100002,Android
1,U100003,Windows
2,U100004,Windows
3,U100005,Android
4,U100006,Android
...,...,...
34045,U136960,Windows
34046,U136961,Android
34047,U136963,Android
34048,U136964,windows


In [44]:
# Merging.
results_data['Most_Active_OS'] = pd.merge(results_data, Most_Active_OS,
                                          how = 'left', on = 'UserID')['Most_Active_OS']
results_data

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS
0,U100002,NaN,2.0,52,pr100258,Android
1,U100003,1.0,2.0,1020,Pr100079,Windows
2,U100004,1.0,15.0,340,Pr100355,Windows
3,U100005,1.0,4.0,680,Pr100234,Android
4,U100006,1.0,1.0,54,Pr101111,Android
...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows
34046,U136961,NaN,NaN,59,Product101,Android
34047,U136963,NaN,NaN,1419,Product101,Android
34048,U136964,NaN,NaN,494,Product101,windows


In [45]:

# 7) Recently_Viewed_Product.
query = 'select UserID,ProductID as Recently_Viewed_Product,max(VisitDateTime)\
    from visitorlogsdata\
    where UserID is not null\
    group by UserID\
    order by UserID\
    ;\
    '

Recently_Viewed_Product = pd.read_sql_query(query, connection)
Recently_Viewed_Product


,UserID,Recently_Viewed_Product,max(VisitDateTime)
0,U100002,pr100258,2018-05-18 10:43:57
1,U100003,None,2018-05-25 02:17:49
2,U100004,pr100142,2018-05-21 13:36:01
3,U100005,Pr101814,2018-05-26 01:06:32
4,U100006,Pr101111,2018-05-21 07:21:59
...,...,...,...
34045,U136960,Pr100172,2018-05-07 00:18:20
34046,U136961,Pr101381,2018-05-07 00:12:09
34047,U136963,pr100102,2018-05-07 00:01:32
34048,U136964,Pr100769,2018-05-07 00:00:31


In [46]:
# Merging.
results_data['Recently_Viewed_Product'] = pd.merge(results_data, Recently_Viewed_Product,
                                                   how = 'left', on = 'UserID')['Recently_Viewed_Product']
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product
0,U100002,NaN,2.0,52,pr100258,Android,pr100258
1,U100003,1.0,2.0,1020,Pr100079,Windows,None
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111
...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769


In [47]:
# Filling NaN values with Product101.
results_data['Recently_Viewed_Product'].fillna('Product101', inplace = True)
results_data

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product
0,U100002,NaN,2.0,52,pr100258,Android,pr100258
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111
...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769


In [48]:

# 8) Pageloads_last_7_days.
query = "select UserID,\
    count(case when upper(activity)='PAGELOAD' then UserID else null end) as Pageloads_last_7_days\
    from visitorlogsdata\
    where VisitDateTime>@current_date-interval 7 day\
    and UserID is not null\
    group by UserID\
    order by userID;\
    "

Pageloads_last_7_days = pd.read_sql_query(query, connection)
Pageloads_last_7_days


,UserID,Pageloads_last_7_days
0,U100003,1
1,U100004,1
2,U100005,1
3,U100006,1
4,U100008,25
...,...,...
16167,U119707,2
16168,U119708,1
16169,U119709,1
16170,U119711,2


In [49]:
# Merging.
results_data['Pageloads_last_7_days'] = pd.merge(results_data, Pageloads_last_7_days,
                                                 how = 'left', on = 'UserID')['Pageloads_last_7_days']
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days
0,U100002,NaN,2.0,52,pr100258,Android,pr100258,NaN
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101,1.0
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142,1.0
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814,1.0
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111,1.0
...,...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172,NaN
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381,NaN
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102,NaN
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769,NaN


In [50]:
# 9) Clicks_last_7_days.
query = "select UserID,\
    count(case when upper(activity)='CLICK' then UserID else null end) as Clicks_last_7_days\
    from visitorlogsdata\
    where VisitDateTime>@current_date-interval 7 day\
    and UserID is not null\
    group by UserID\
    order by userID;\
    "

Clicks_last_7_days = pd.read_sql_query(query, connection)
Clicks_last_7_days


,UserID,Clicks_last_7_days
0,U100003,2
1,U100004,0
2,U100005,0
3,U100006,0
4,U100008,28
...,...,...
16167,U119707,9
16168,U119708,0
16169,U119709,0
16170,U119711,0


In [51]:
# Merging.
results_data['Clicks_last_7_days'] = pd.merge(results_data, Clicks_last_7_days,
                                              how = 'left', on = 'UserID')['Clicks_last_7_days']
results_data


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,NaN,2.0,52,pr100258,Android,pr100258,NaN,NaN
1,U100003,1.0,2.0,1020,Pr100079,Windows,Product101,1.0,2.0
2,U100004,1.0,15.0,340,Pr100355,Windows,pr100142,1.0,0.0
3,U100005,1.0,4.0,680,Pr100234,Android,Pr101814,1.0,0.0
4,U100006,1.0,1.0,54,Pr101111,Android,Pr101111,1.0,0.0
...,...,...,...,...,...,...,...,...,...
34045,U136960,NaN,NaN,754,Product101,Windows,Pr100172,NaN,NaN
34046,U136961,NaN,NaN,59,Product101,Android,Pr101381,NaN,NaN
34047,U136963,NaN,NaN,1419,Product101,Android,pr100102,NaN,NaN
34048,U136964,NaN,NaN,494,Product101,windows,Pr100769,NaN,NaN


In [52]:
# Exporting final results.
results_data.to_csv('ETL_solution.csv', index = False)